In [9]:
import os
import pydicom
from PIL import Image
import pandas as pd 


In [10]:
df=pd.read_csv('../csv_data/222.csv')
df.head()

,住院号,姓名,性别,年龄,入院时体温,心率,收缩压,舒张压,体重,身高,...,3V/4pai.1,2/3次方2,等v2的s,SR2,真正的SF2,vequS1（等体积的球的表面积）,vequS2,sd1,sd2,SF差值
0,821657.0,刘继军,1.0,61.0,36.5,84.0,180.0,117.0,NaN,NaN,...,2353.285794,176.922519,2223.273841,0.662427,0.759904,2219.090338,2223.273877,702.559662,7634.146123,0.000371
1,822545.0,胡生朗,1.0,75.0,36.8,60.0,134.0,82.0,NaN,NaN,...,8198.047646,406.574646,5109.167428,0.489897,0.621446,5031.214997,5109.167510,2552.505003,29230.732490,-0.041977
2,822853.0,康小玉,2.0,56.0,36.9,87.0,145.0,87.0,68.0,NaN,...,1642.259459,139.196350,1749.192839,0.692638,0.782836,1636.142203,1749.192867,367.987797,5129.887133,-0.033549
3,825791.0,强光荣,1.0,71.0,36.6,90.0,168.0,101.0,78.0,NaN,...,2638.375283,190.937418,2399.390247,0.222695,0.367401,2274.847854,2399.390285,1639.082146,8652.209715,-0.213818
4,827598.0,李建芳,2.0,66.0,36.8,84.0,121.0,81.0,NaN,NaN,...,3001.415683,208.073816,2614.732557,0.650622,0.750849,2505.619115,2614.732599,762.220885,9957.567401,-0.015902


In [11]:
df.shape

(251, 126)

In [12]:
df = df.dropna(subset=['住院号'])
df['住院号'] = df['住院号'].astype(int)
selected_columns = df[['住院号', '姓名','性别','脑出血位置1']]


In [13]:
selected_columns.head(20)

,住院号,姓名,性别,脑出血位置1
0,821657,刘继军,1.0,Subtentorial
1,822545,胡生朗,1.0,Subtentorial
2,822853,康小玉,2.0,Deep
3,825791,强光荣,1.0,Deep
4,827598,李建芳,2.0,Deep
5,828450,朱津亮,1.0,Deep
6,828491,王玉梅,2.0,Deep
7,830666,张海兰,2.0,Subtentorial
8,831291,白建章,1.0,Deep
9,831377,李凤卫,2.0,Deep


In [14]:
selected_columns[selected_columns['住院号']==int(737921)]

,住院号,姓名,性别,脑出血位置1
176,737921,魏永成,1.0,Lobar


In [15]:
# 手动数据补全，从9.26数据大合里挑
new_data=([{'住院号': 800382, '姓名': '王平', '性别':2,'脑出血位置1': 'Subtentorial'},
                         {'住院号': 776498, '姓名': '竹花','性别':2, '脑出血位置1': 'Lobar'},
                         ])
new_df = pd.DataFrame(new_data)
selected_columns = pd.concat([selected_columns,new_df],ignore_index=True)



In [16]:
# get id based on name 

selected_columns[selected_columns['姓名']=='李小琴']

,住院号,姓名,性别,脑出血位置1
13,832578,李小琴,2.0,Lobar


## 统计四年脑出血病人的类型，以及每个类型的数量，保存在文件夹下

In [14]:
# process yes
from tqdm import tqdm
import shutil
from pypinyin import lazy_pinyin
from collections import defaultdict
base_dir="/home/jialiangfan/dataset/"

yes_images_list=['2018_filtered','2019_filtered','2020_filtered','2021_filtered']
save_dir='../dataset/yes'
category_dir=['Subtentorial','Deep','Lobar']
Subtentorial_num=0
deep_num=0
lobar_num=0
total_num=0
total_patient_num=0
deep_patient_num=0
lobar_patient_num=0
Subtentorial_patient_num=0
patient_dict=defaultdict(list)
patient_sex_dict=defaultdict(list)
if os.path.exists(save_dir):
    shutil.rmtree(save_dir)  # 删除整个目录树
os.makedirs(save_dir)  # 重新创建目录
for i in range(3):
    os.makedirs(os.path.join(save_dir,category_dir[i]))
counter=0
not_found=set()
male_num=0
female_num=0
for yes_images in yes_images_list:
    for root, dirs, files in os.walk(base_dir+yes_images):
        for file in files:
            if file.endswith('.jpg'):
                file_path = os.path.join(root, file)
                print('file_paths:', file_path)
                dir_name=root.split('/')[-2]
                dir_name=dir_name.split(' ')
                # print('dir_name:',dir_name)
                patient_name,patient_id=dir_name[0],dir_name[-1]
                # print("patient id:",patient_id) 
                try:
                    category=selected_columns[selected_columns['住院号']==int(patient_id)]['脑出血位置1'].values[0]
                    sex=selected_columns[selected_columns['住院号']==int(patient_id)]['性别'].values[0]
                    if int(sex)==1:
                        male_num+=1
                        patient_sex_dict['male'].append(patient_id)
                    elif int(sex)==2:
                        patient_sex_dict['female'].append(patient_id)
                        female_num+=1
                    # {'Subtentorial': [], 'Deep': [], 'Lobar': []}
                    patient_dict[category].append(patient_id)
                    if category=='Subtentorial':
                        Subtentorial_num+=1
                    elif category=='Deep':
                        deep_num+=1
                    elif category=='Lobar':
                        lobar_num+=1
                    else:
                        print('Program error:'.format(category))
                    total_num+=1
                except IndexError:
                    # print('patient {} not found'.format(patient_id))
                    not_found.add((patient_name,patient_id))
                    continue
                en_name=''.join(lazy_pinyin(patient_name))
                new_file_path = os.path.join(os.path.join(save_dir,category), '%04d-{}-{}-{}.jpg'.format(yes_images,en_name,patient_id) % counter)
                shutil.copy(file_path, new_file_path)
                print('copy {} file from {} to {}'.format(counter,file_path, new_file_path))
                counter+=1



file_paths: /home/jialiangfan/dataset/2018_filtered/1润花 724010/1/Image74.jpg
copy 0 file from /home/jialiangfan/dataset/2018_filtered/1润花 724010/1/Image74.jpg to ../dataset/yes/Lobar/0000-2018_filtered-1runhua-724010.jpg
file_paths: /home/jialiangfan/dataset/2018_filtered/1润花 724010/1/Image56.jpg
copy 1 file from /home/jialiangfan/dataset/2018_filtered/1润花 724010/1/Image56.jpg to ../dataset/yes/Lobar/0001-2018_filtered-1runhua-724010.jpg
file_paths: /home/jialiangfan/dataset/2018_filtered/1润花 724010/1/Image55.jpg
copy 2 file from /home/jialiangfan/dataset/2018_filtered/1润花 724010/1/Image55.jpg to ../dataset/yes/Lobar/0002-2018_filtered-1runhua-724010.jpg
file_paths: /home/jialiangfan/dataset/2018_filtered/1润花 724010/1/Image65.jpg
copy 3 file from /home/jialiangfan/dataset/2018_filtered/1润花 724010/1/Image65.jpg to ../dataset/yes/Lobar/0003-2018_filtered-1runhua-724010.jpg
file_paths: /home/jialiangfan/dataset/2018_filtered/1润花 724010/1/Image60.jpg
copy 4 file from /home/jialiangfan/data

ValueError: invalid literal for int() with base 10: '1.2.840.113704.1.111.7752.1535945133.1'

In [13]:
not_found

{('1', '1'),
 ('1世丁', '749236'),
 ('1凤萍', '722419'),
 ('1占伟', '728999'),
 ('1延保', '737698'),
 ('1建香', '730496'),
 ('1彩艳', '741085'),
 ('1慧玲', '739139'),
 ('1文泽', '735142'),
 ('1桂兰', '730601'),
 ('1永库', '743209'),
 ('1玉彪', '737046'),
 ('1生义', '739432'),
 ('1福军', '727442'),
 ('2', '2')}

In [14]:
# 88个病人没有找到
len(not_found)

15

In [15]:
print("total_num:",total_num)
print("Subtentorial_num:",Subtentorial_num,"percentage:",round(Subtentorial_num/total_num,4))
print("deep_num:",deep_num,"percentage:",deep_num/total_num,round(deep_num/total_num,4))
print("lobar_num:",lobar_num,"percentage:",lobar_num/total_num,round(lobar_num/total_num,4))

total_num: 1570
Subtentorial_num: 37 percentage: 0.0236
deep_num: 1091 percentage: 0.6949044585987261 0.6949
lobar_num: 442 percentage: 0.28152866242038216 0.2815


In [16]:
Subtentorial_patient_num=len(set(patient_dict['Subtentorial']))
deep_patient_num=len(set(patient_dict['Deep']))
lobar_patient_num=len(set(patient_dict['Lobar']))
total_patient_num=Subtentorial_patient_num+deep_patient_num+lobar_patient_num
print("total_patient_num:",total_patient_num)
print("male num",male_num,"female num",female_num)
print("male patient num",len(set(patient_sex_dict['male'])))
print("female patient num", len(set(patient_sex_dict['female'])))
print("Subtentorial_patient_num:",Subtentorial_patient_num)
print("deep_patient_num:",deep_patient_num)
print("lobar_patient_num:",lobar_patient_num)

total_patient_num: 50
male num 884 female num 686
male patient num 27
female patient num 23
Subtentorial_patient_num: 1
deep_patient_num: 38
lobar_patient_num: 11


## 处理正常病人的数据

In [14]:
# process no
from tqdm import tqdm
import shutil

images='/home/jialiangfan/dataset/dcm2images_filtered'
save_dir='../dataset/no/'
if os.path.exists(save_dir):
    shutil.rmtree(save_dir)  # 删除整个目录树
os.makedirs(save_dir)  # 重新创建目录
counter=0
patient_dict=defaultdict(list)
for root, dirs, files in os.walk(images):
    for file in files:
        # dir_name=dir_name.split(' ')
        # print('dir_name:',dir_name)
        # patient_name,patient_id=dir_name[0],dir_name[-1]
        # print("patient id:",patient_id)
        # print("patient name:",patient_name)
        if file.endswith('.jpg'):
            file_path = os.path.join(root, file)
            dir_name=root.split('/')[-1]
            patient_dict[dir_name].append(file_path)
            # en_name=''.join(lazy_pinyin(dir_name))
            new_file_path = os.path.join(save_dir, '%04d-{}.jpg'.format(dir_name) % counter)
            shutil.copy(file_path, new_file_path)
            # print('copy {} file from {} to {}'.format(counter,file_path, new_file_path))
            counter+=1
print('total {} images'.format(counter))
print('total {} patients'.format(len(patient_dict.keys())))

total 4407 images
total 200 patients


In [15]:
import os

def count_jpg_images(folder_path):
    count = 0
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith('.jpg'):
                count += 1
    return count

folder_path = '../dataset2/yes/'  # Replace with the actual folder path
jpg_count = count_jpg_images(folder_path)
print(f"Number of JPG images claasified yes in {folder_path}: {jpg_count}")

folder_path = '../dataset2/no/'  # Replace with the actual folder path
jpg_count = count_jpg_images(folder_path)
print(f"Number of JPG images claasified no  in {folder_path}: {jpg_count}")

deep_images='../dataset2/yes/Deep'
subtentorial_images='../dataset2/yes/Subtentorial'
lobar_images='../dataset2/yes/Lobar'
print('deep:',len(os.listdir(deep_images)))
print('subtentorial:',len(os.listdir(subtentorial_images)))
print('lobar:',len(os.listdir(lobar_images)))


Number of JPG images claasified yes in ../dataset2/yes/: 8244
Number of JPG images claasified no  in ../dataset2/no/: 4407
deep: 6093
subtentorial: 495
lobar: 1656
